In [ ]:
!wget https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz

--2023-12-07 19:04:44--  https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.12.207, 108.177.13.207, 74.125.26.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.12.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228813984 (218M) [application/x-compressed-tar]
Saving to: ‘flower_photos.tgz’

flower_photos.tgz   100%[===================>] 218.21M   234MB/s    in 0.9s    

2023-12-07 19:04:45 (234 MB/s) - ‘flower_photos.tgz’ saved [228813984/228813984]



In [ ]:
!pip install mahotas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 42.3 MB/s eta 0:00:00


In [ ]:
!tar -xzvf "flower_photos.tgz" -C "." > xx

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import os
import sklearn

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, cohen_kappa_score
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, concatenate

In [ ]:
def load_img(indir):
    samples = []
    labels = []
    for class_dir in os.listdir(indir):
        if not os.path.isdir(indir+'/'+class_dir):
          continue
        print("Loading:",class_dir)
        the_class = class_dir
        for file in os.listdir(indir+'/'+class_dir):
            image = cv2.imread("{}/{}/{}".format(indir,class_dir,file))
            image = cv2.resize(image, (64,64))
            samples.append(image)
            labels.append(the_class)
    samples = np.array(samples)
    labels = np.array(labels)
    return samples,labels
samples, labels = load_img('flower_photos')
print('loaded',len(samples),' samples')
print('classes',set(labels))
org_samples = samples
org_labels = labels

Loading: daisy
Loading: sunflowers
Loading: dandelion
Loading: .ipynb_checkpoints
Loading: tulips
Loading: roses
loaded 3670  samples
classes {'sunflowers', 'daisy', 'dandelion', 'roses', 'tulips'}


In [ ]:

encoder = sklearn.preprocessing.LabelEncoder() # encoder
intlabels = encoder.fit_transform(labels)
labels = tf.keras.utils.to_categorical(intlabels)
print("One-hot labels",labels[:657])

## find text label for label
label = np.array([0,1])
print('Label:',label)
print("Text label:",encoder.inverse_transform(label)) # WRONG! each value treated independently!
print('argmax label:',label.argmax())
#print(encoder.inverse_transform(label.argmax())) # encoder expects colection of values!
print("Text label:",encoder.inverse_transform([label.argmax()])) # works when packed into a collection

One-hot labels [[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]]
Label: [0 1]
Text label: ['daisy' 'dandelion']
argmax label: 1
Text label: ['dandelion']


In [ ]:
model = Sequential()
model.add(Conv2D(16, (3, 3), padding="same",input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dense(4))
model.add(Activation("softmax"))

model.summary()
optimizer = tf.keras.optimizers.RMSprop(0.0001)
#model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 64, 64, 16)        448       
                                                                 
 batch_normalization_4 (Bat  (None, 64, 64, 16)        64        
 chNormalization)                                                
                                                                 
 activation_8 (Activation)   (None, 64, 64, 16)        0         
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 32, 32, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 32, 32, 32)        4640      
                                                                 
 batch_normalization_5 (Bat  (None, 32, 32, 32)       

In [ ]:
image_size = tuple((64, 64))

def load_img(indir):
    smp = []
    labels = []
    for class_dir in os.listdir(indir):
        the_class = class_dir
        for file in os.listdir(indir+'/'+class_dir):
            image = cv2.imread("{}/{}/{}".format(indir,class_dir,file))
            image = cv2.resize(image, image_size)
            smp.append(image)
            labels.append(the_class)
    smp = np.array(smp)
    labels = np.array(labels)
    return smp,labels
smp, labels = load_img('flower_photos')
smp,labels = sklearn.utils.shuffle(smp,labels)
print('loaded',len(smp),' samples')
print('classes',set(labels))

loaded 3670  samples
classes {'sunflowers', 'daisy', 'dandelion', 'roses', 'tulips'}


In [ ]:
import mahotas


# feature-descriptor-1: Hu Moments
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

def fd_haralick(image):
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick

def fd_histogram(image, mask=None):
    bins=8
    # convert the image to HSV color-space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    return hist.flatten()

In [ ]:
new_samples = []
for i,image in enumerate(smp):
    fv_hu_moments = fd_hu_moments(image)
    fv_haralick   = fd_haralick(image)
    fv_histogram  = fd_histogram(image)
    if(i%500==0): print(i,'/',len(smp))
    features = np.hstack([fv_histogram, fv_haralick, fv_hu_moments])
    new_samples.append(features)
smp = np.array(new_samples)
#np.savetxt('samples.csv',fsamples) #save it if you want to reuse it!
print(smp.shape)

0 / 3670
500 / 3670
1000 / 3670
1500 / 3670
2000 / 3670
2500 / 3670
3000 / 3670
3500 / 3670
(3670, 532)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
print(np.min(smp),np.max(smp))
smp = scaler.fit_transform(smp)
print(np.min(smp),np.max(smp))


-0.6241640388357443 42116.99917305573
0.0 1.0000000000000004


In [ ]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = np.array(labels,dtype=float)
print(labels.shape)

(3670, 5)


In [ ]:
dense_model = Sequential()
dense_model.add(Dense(250, input_dim=532, activation='relu'))
dense_model.add(Dense(250, activation='relu'))
dense_model.add(Dense(250, activation='relu'))
dense_model.add(Dense(5, activation='softmax'))
dense_model.summary()
#dense_model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 250)               133250    
                                                                 
 dense_13 (Dense)            (None, 250)               62750     
                                                                 
 dense_14 (Dense)            (None, 250)               62750     
                                                                 
 dense_15 (Dense)            (None, 5)                 1255      
                                                                 
Total params: 260005 (1015.64 KB)
Trainable params: 260005 (1015.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#from tensorflow.python.keras.models import Model
Model = tf.keras.models.Model

import numpy as np
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Sequential

In [ ]:
combined = concatenate([model.output, dense_model.output])
combined = Dense(16, activation="sigmoid")(combined)
combined = Dense(5, activation="sigmoid")(combined)

mod = Model(inputs=[model.input, dense_model.input], outputs=combined)

print(mod.summary())

mod.compile(loss='categorical_crossentropy', optimizer="adam",metrics=['accuracy'])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 conv2d_4_input (InputLayer  [(None, 64, 64, 3)]          0         []                            
 )                                                                                                
                                                                                                  
 conv2d_4 (Conv2D)           (None, 64, 64, 16)           448       ['conv2d_4_input[0][0]']      
                                                                                                  
 batch_normalization_4 (Bat  (None, 64, 64, 16)           64        ['conv2d_4[0][0]']            
 chNormalization)                                                                                 
                                                                                              

In [ ]:
EPOCHS = 10
BATCH=100
mod.fit([samples,smp], labels, batch_size=BATCH, epochs=EPOCHS)

Epoch 1/10


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
37/37 [==============================] - 23s 522ms/step - loss: 1.6330 - accuracy: 0.2177
Epoch 2/10
37/37 [==============================] - 19s 514ms/step - loss: 1.5717 - accuracy: 0.3153
Epoch 3/10
37/37 [==============================] - 21s 577ms/step - loss: 1.5490 - accuracy: 0.3221
Epoch 4/10
37/37 [==============================] - 18s 498ms/step - loss: 1.5304 - accuracy: 0.3332
Epoch 5/10
37/37 [==============================] - 18s 497ms/step - loss: 1.5094 - accuracy: 0.4245
Epoch 6/10
37/37 [==============================] - 19s 514ms/step - loss: 1.4865 - accuracy: 0.4632
Epoch 7/10
37/37 [==============================] - 20s 522ms/

In [ ]:
results1 = mod.predict([samples,smp])
print(confusion_matrix(labels.argmax(axis=1), results1.argmax(axis=1)))
print(classification_report(labels.argmax(axis=1), results1.argmax(axis=1)))
print("Accuracy: {:.2f}".format(accuracy_score(labels.argmax(axis=1), results1.argmax(axis=1))))
print("Cohen's Kappa {:.2f}".format(cohen_kappa_score(labels.argmax(axis=1), results1.argmax(axis=1))))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
115/115 [==============================] - 7s 55ms/step
[[  0 382  16 194  41]
 [  0 761  31  90  16]
 [  0 128 290  36 187]
 [  0  67   0 603  29]
 [  0 101  89 113 496]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       633
           1       0.53      0.85      0.65       898
           2       0.68      0.45      0.54       641
           3       0.58      0.86      0.70       699
           4       0.64      0.62      0.63       799

    accuracy                           0.59      3670
   macro avg       0.49      0.56      0.50      3670
weighted avg       0.50      0.59      0.52      36

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
